In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras import layers

In [33]:
train_data_all = pd.read_csv("../data/train.csv")

In [34]:
train_data_all.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
# replace missing values in training data

train_data_filtered = pd.DataFrame()

for column in train_data_all.columns:
    dtype = train_data_all[column].dtype

    # use -1 for numerical data, "NA" for strings (objects)
    if dtype == "int64" or dtype == "float64":
        val = -1
    else:
        val = "NA"
    train_data_filtered[column] = train_data_all[column].fillna(val)

# ignore names (for now)
train_data_filtered.drop(columns="Name", inplace=True)

In [36]:
# split into training and validation datasets

val_frac = .2
val_df = train_data_filtered.sample(frac=0.2)
train_df = train_data_filtered.drop(val_df.index)

In [37]:
# convert pd DataFrame to tf Dataset labelled with answers
def dataframe_to_dataset(df):
    
    df = df.copy()
    labels = df.pop("Survived")
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))

    return ds

In [38]:
train_ds = dataframe_to_dataset(train_df)
val_ds = dataframe_to_dataset(val_df)